In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'lambda': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
                         
            # sample selection
            'alpha':0.01
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_lambda={params['lambda']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=MNIST/seed=1_epoch=1_lr=0.001_alpha=0.01_lambda=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = min(params['per_task_examples'], 10000),
                      random_class_idx = True)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = min(params['per_task_examples'], 15000),
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28

elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = cl.benchmarks.SplitCIFAR10(num_tasks=params['num_tasks'],
                                        per_task_memory_examples=params['per_task_memory_examples'])
    n_feature = 32*32*3



[2 9 6 4 0 3 1 7 8 5]


In [3]:
benchmark.trains[1].inputs.shape

torch.Size([11907, 1, 28, 28])

In [4]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [5]:
from algorithms import GreedySelection
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = GreedySelection(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [6]:
from trainers import ContinualTrainer2

trainer = ContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())



---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 97.64901737079464, 'loss': 0.00030053448386486694, 'std': 0.07149799094967046}
training_task_end
---------------------------- Task 2 -----------------------
len(select_curr_indexes)=1446
[2] Eval metrics for task 1 >> {'accuracy': 94.1716892924916, 'loss': 0.001025001276129312, 'std': 4.539906346755168}
[2] Eval metrics for task 2 >> {'accuracy': 4.801670146137787, 'loss': 0.008308703506115785, 'std': 4.801670146137787}
training_task_end
---------------------------- Task 3 -----------------------
len(select_curr_indexes)=769
[3] Eval metrics for task 1 >> {'accuracy': 85.24577254323492, 'loss': 0.002429640684450216, 'std': 8.113989597498483}
[3] Eval metrics for task 2 >> {'accuracy': 78.2457938083839, 'loss': 0.003236890484377281, 'std': 16.53499951103155}
[3] Eval metrics for task 3 >> {'accuracy': 0.0, 'loss': 0.0108034818016704, 'std': 0.0}
training_task_end
---------------------

In [7]:
metric_manager_callback.meters['std'].get_std()

[0.07149799094967046,
 44.928648694009496,
 40.077667368830824,
 36.00566791248204,
 33.618563143145025]

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[97.649,  0.   ,  0.   ,  0.   ,  0.   ],
       [94.172,  4.802,  0.   ,  0.   ,  0.   ],
       [85.246, 78.246,  0.   ,  0.   ,  0.   ],
       [73.206, 83.169, 85.405,  0.   ,  0.   ],
       [71.776, 88.6  , 76.85 , 88.92 ,  0.   ]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

[97.64901737079464,
 49.486679719314694,
 81.74578317580941,
 80.59316183543541,
 81.53638329434854]